In [ ]:
from sqlalchemy import create_engine

In [ ]:
def connection_yandex_cloud_demo(echo):
    """Connection to DataBase dwh"""
    login = 'root'
    password = 'password!!!'
    host = 'rc1b-7ng6ih3jte3824x8.mdb.yandexcloud.net'
    port = '6432'
    db_name = 'demo'
    return create_engine(f'postgresql://{login}:{password}@{host}:{port}/{db_name}', echo=echo)

In [ ]:
engine = connection_yandex_cloud_demo(echo=True)

**Оптимизация с помощью индекса**

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.tickets t
  WHERE t.passenger_id = '7028 453406'
'''

engine.execute(sql).fetchall()

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.tickets t
  WHERE t.passenger_id like '%%453406'
'''

engine.execute(sql).fetchall()

**Создаём индекс на атрибут bookings.tickets.passenger_id**

In [ ]:
sql = '''
CREATE 
 INDEX tickets_passenger_id_idx 
    ON bookings.tickets (passenger_id)
'''

engine.execute(sql)

**Выполним запросы после создания индекса**

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.tickets t
  WHERE t.passenger_id = '7028 453406'
'''

engine.execute(sql).fetchall()

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.tickets t
  WHERE t.passenger_id like '%%453406'
'''

engine.execute(sql).fetchall()

**Выполним запрос к таблице flights и сделаем фильтрацию по двум атрибутам**

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.flights f
  WHERE f.departure_airport = 'DME'
    AND f.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

**Создадим составной индекс**

In [ ]:
sql = '''
CREATE 
 INDEX flights_departure_airport_idx 
    ON bookings.flights (departure_airport,arrival_airport)
'''

engine.execute(sql)

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.flights f
  WHERE f.departure_airport != 'DME'
    AND f.arrival_airport = 'LED'
'''

engine.execute(sql).fetchall()

**Допустим, нам необходимо забирать данные за предыдущий день о совершенных полётах**

In [ ]:
sql = '''
EXPLAIN (ANALYZE)
 SELECT * 
   FROM bookings.flights f 
  WHERE DATE(f.actual_departure) = '2017-09-06'
'''

engine.execute(sql).fetchall()

**Создадим индекс на колонку с датой отправления**

In [ ]:
sql = '''
CREATE INDEX flights_actual_departure_idx ON bookings.flights (actual_departure)
'''

engine.execute(sql)

**Проверим, ускорился ли наш запрос**

In [ ]:
sql = '''
EXPLAIN (ANALYZE)
 SELECT * 
   FROM bookings.flights f 
  WHERE DATE(f.actual_departure) > '2017-09-06'
'''

engine.execute(sql).fetchall()

**UNION vs UNION ALL**

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.flights f
  WHERE f.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION
 SELECT * 
   FROM bookings.flights f2
  WHERE f2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

In [ ]:
sql = '''
EXPLAIN (ANALYZE) 
 SELECT * 
   FROM bookings.flights f
  WHERE f.scheduled_departure < '2017-01-01 00:00:00.000 +0300'
 UNION ALL
 SELECT * 
   FROM bookings.flights f2
  WHERE f2.scheduled_departure >= '2017-01-01 00:00:00.000 +0300'
'''

engine.execute(sql).fetchall()

**Временные таблицы**

In [ ]:
sql = '''
EXPLAIN (ANALYSE)
SELECT f.*
      ,tf.ticket_no
      ,tf.fare_conditions 
      ,tf.amount 
FROM bookings.flights f
JOIN bookings.ticket_flights tf ON tf.flight_id = f.flight_id 
'''

engine.execute(sql).fetchall()

**Создаём временную таблицу**

In [ ]:
sql = '''
CREATE TEMPORARY TABLE ticket_flights_materialized AS 
SELECT f.*
      ,tf.ticket_no
      ,tf.fare_conditions 
      ,tf.amount 
  FROM bookings.flights f
  JOIN bookings.ticket_flights tf ON tf.flight_id = f.flight_id;
'''

engine.execute(sql).fetchall()

**Сделаем запрос к веременной таблице**

In [ ]:
sql = '''
EXPLAIN (ANALYSE)
 SELECT tfm.*
   FROM ticket_flights_materialized tfm;
'''

engine.execute(sql).fetchall()